## In this Notebook, I will demo how to use the FTXDataProcessor class to fetch historical data of perpetual and exipred futures data and calculate historical spreads. 

In [1]:
from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime, timedelta
import time
import hmac
import pandas as pd
from requests import Request, Session, Response
from typing import Optional, Dict, Any, List
from utility_classes.historical_data_processor import FTXDataProcessor
import dateutil.parser as dp
import json
import matplotlib.pyplot as plt

acc = FTXDataProcessor(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)


In [2]:
(datetime.now() - timedelta(hours = 1)).timestamp()

1617857276.598931

## get all tokens that have perpetual contracts

In [3]:

current_perp_futures = acc.get_all_perp_tickers()
print(current_perp_futures)


['1INCH', 'AAVE', 'ADA', 'ALCX', 'ALGO', 'ALPHA', 'ALT', 'AMPL', 'AR', 'ATOM', 'AUDIO', 'AVAX', 'AXS', 'BADGER', 'BAL', 'BAND', 'BAO', 'BAT', 'BCH', 'BNB', 'BNT', 'BRZ', 'BSV', 'BTC', 'BTMX', 'BTT', 'CAKE', 'CHZ', 'COMP', 'CONV', 'CREAM', 'CRO', 'CRV', 'CUSDT', 'DASH', 'DAWN', 'DEFI', 'DENT', 'DMG', 'DODO', 'DOGE', 'DOT', 'DRGN', 'EGLD', 'ENJ', 'EOS', 'ETC', 'ETH', 'EXCH', 'FIDA', 'FIL', 'FLM', 'FLOW', 'FTM', 'FTT', 'GRT', 'HBAR', 'HNT', 'HOLY', 'HOT', 'HT', 'HUM', 'IOTA', 'KAVA', 'KIN', 'KNC', 'KSM', 'LEO', 'LINA', 'LINK', 'LRC', 'LTC', 'LUNA', 'MAPS', 'MATIC', 'MID', 'MKR', 'MTA', 'MTL', 'NEAR', 'NEO', 'OKB', 'OMG', 'ONT', 'ORBS', 'OXY', 'PAXG', 'PERP', 'PRIV', 'PUNDIX', 'QTUM', 'RAY', 'REEF', 'REN', 'ROOK', 'RSR', 'RUNE', 'SAND', 'SC', 'SECO', 'SHIT', 'SKL', 'SNX', 'SOL', 'SRM', 'SRN', 'STMX', 'STORJ', 'STX', 'SUSHI', 'SXP', 'THETA', 'TOMO', 'TRU', 'TRX', 'TRYB', 'UNI', 'UNISWAP', 'USDT', 'VET', 'WAVES', 'XAUT', 'XEM', 'XLM', 'XMR', 'XRP', 'XTZ', 'YFI', 'YFII', 'ZEC', 'ZIL', 'ZRX']


## getting symbols for all expired futures

In [4]:
expired_futures = acc._get('expired_futures')
for ticker in expired_futures: 
    print(ticker['name'], ticker['expiryDescription'])


01130 2020-11-30
BTC-MOVE-20201129 2020-11-29
BTC-MOVE-20201128 2020-11-28
BTC-MOVE-20201127 2020-11-27
BTC-MOVE-WK-20201127 2020-11-27
BTC-MOVE-20201126 2020-11-26
BTC-MOVE-20201125 2020-11-25
BTC-MOVE-20201124 2020-11-24
BTC-MOVE-20201123 2020-11-23
BTC-MOVE-20201122 2020-11-22
BTC-MOVE-20201121 2020-11-21
BTC-MOVE-20201120 2020-11-20
BTC-MOVE-WK-20201120 2020-11-20
BTC-MOVE-20201119 2020-11-19
BTC-MOVE-20201118 2020-11-18
BTC-MOVE-20201117 2020-11-17
BTC-MOVE-20201116 2020-11-16
BTC-MOVE-20201115 2020-11-15
BTC-MOVE-20201114 2020-11-14
BTC-MOVE-WK-20201113 2020-11-13
BTC-MOVE-20201113 2020-11-13
BTC-MOVE-20201112 2020-11-12
BTC-MOVE-20201111 2020-11-11
BTC-MOVE-20201110 2020-11-10
BTC-MOVE-20201109 2020-11-09
BTC-MOVE-20201108 2020-11-08
BTC-MOVE-20201107 2020-11-07
BTC-MOVE-WK-20201106 2020-11-06
BTC-MOVE-20201106 2020-11-06
BTC-MOVE-20201105 2020-11-05
BTC-MOVE-20201104 2020-11-04
BTC-MOVE-20201103 2020-11-03
BTC-MOVE-20201102 2020-11-02
OIL100-20201026 2020-10-26
BTC-MOVE-2020110

## To calculate spreads, I need to make sure that all the futures fetched have perpetual contract counterparts

In [5]:
expired_futures=acc.get_all_expired_futures_that_have_perps()
expired_futures.sort()
print(expired_futures)

['1INCH', 'AAVE', 'ADA', 'ALGO', 'ALT', 'ATOM', 'AVAX', 'BAL', 'BCH', 'BNB', 'BRZ', 'BSV', 'BTC', 'BTMX', 'CHZ', 'COMP', 'CREAM', 'CUSDT', 'DEFI', 'DMG', 'DOGE', 'DOT', 'DOTPRESPLIT', 'DRGN', 'EOS', 'ETC', 'ETH', 'EXCH', 'FIL', 'FLM', 'GRT', 'HNT', 'HT', 'KNC', 'LEO', 'LINK', 'LTC', 'MATIC', 'MID', 'MKR', 'MTA', 'NEO', 'OKB', 'OMG', 'PAXG', 'PRIV', 'RUNE', 'SHIT', 'SOL', 'SUSHI', 'SXP', 'THETA', 'TOMO', 'TRU', 'TRX', 'TRYB', 'UNI', 'UNISWAP', 'USDT', 'VET', 'WAVES', 'XAUT', 'XRP', 'XTZ', 'YFI', 'ZEC']


## all past futures expiration dates format
### to get past data requires correct date input 
### *only futures data after 20190628 is available on FTX

In [6]:
acc.get_expired_futures_dates()


expiration date: 0326 | Date March 2021
expiration date: 20201225 | Date December 2020
expiration date: 20200925 | Date September 2020
expiration date: 20200626 | Date June 2020
expiration date: 20200327 | Date March 2020
expiration date: 20191227 | Date December 2019
expiration date: 20190927 | Date September 2019
expiration date: 20190628 | Date June 2019
expiration date: 20190329 | Date March 2019
BTC-0626 data is missing from FTX end


## To get an expired futures contract given specific expired date
### *BTC futures have longer periods than alts


In [7]:
alt_hist_futures = acc.get_expired_futures_OHCL(market = 'ETH-0925', resolution = 3600)
btc_hist_futures = acc.get_expired_futures_OHCL(market = 'BTC-0925', resolution = 3600)
print(alt_hist_futures)
print(btc_hist_futures)

Exception: No such market: ETH-0925 

## get all historical perp contract funding rate

In [8]:
acc.get_all_historical_funding_rates(path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/historical_funding')


Adding 1 funding data with start time 2020-08-31 18:00:00
Adding 500 funding data with start time 2021-03-18 06:00:00
Adding 500 funding data with start time 2021-02-25 10:00:00
Adding 500 funding data with start time 2021-02-04 15:00:00
Adding 500 funding data with start time 2021-01-14 20:00:00
Adding 500 funding data with start time 2020-12-25 01:00:00
Adding 500 funding data with start time 2020-12-04 06:00:00
Adding 500 funding data with start time 2020-11-13 11:00:00
Adding 500 funding data with start time 2020-10-23 17:00:00
Adding 500 funding data with start time 2020-10-02 22:00:00
Adding 500 funding data with start time 2020-09-12 03:00:00
Adding 500 funding data with start time 2020-08-22 08:00:00
Adding 500 funding data with start time 2020-08-01 13:00:00
Adding 474 funding data with start time 2020-07-12 20:00:00
Adding 1 funding data with start time 2020-07-12 20:00:00
Adding 500 funding data with start time 2021-03-18 07:00:00
Adding 500 funding data with start time 2021

In [2]:
acc.write_all_PERPs_OHCL(path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/all_perps')


les with start time 2019-09-10 12:12:00
Adding 4997 candles with start time 2019-09-07 00:52:00
Adding 5001 candles with start time 2019-09-03 13:32:00
Adding 5001 candles with start time 2019-08-31 02:12:00
Adding 5001 candles with start time 2019-08-27 14:52:00
Adding 5001 candles with start time 2019-08-24 03:32:00
Adding 5001 candles with start time 2019-08-20 16:12:00
Adding 4994 candles with start time 2019-08-17 04:52:00
Adding 5001 candles with start time 2019-08-13 17:32:00
Adding 5001 candles with start time 2019-08-10 06:12:00
Adding 5001 candles with start time 2019-08-06 18:52:00
Adding 5001 candles with start time 2019-08-03 07:32:00
Adding 4999 candles with start time 2019-07-30 20:12:00
Adding 4918 candles with start time 2019-07-27 08:52:00
Adding 5001 candles with start time 2019-07-23 21:32:00
Adding 98 candles with start time 2019-07-20 10:12:00
Adding 4999 candles with start time 2021-04-08 21:38:25
Adding 5001 candles with start time 2021-04-05 10:19:00
Adding 500

## fetching all expired futures and write them into csv


In [3]:
acc.write_all_expired_futures_OHCL(path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/expired_futures_data', resolution=60)


There has been an error in writing file BTC
There has been an error in writing file AVAX
There has been an error in writing file ETH
There has been an error in writing file EOS
There has been an error in writing file BRZ
There has been an error in writing file DOGE
There has been an error in writing file AAVE
There has been an error in writing file UNISWAP
There has been an error in writing file TRX
There has been an error in writing file PRIV
There has been an error in writing file USDT
There has been an error in writing file BNB
There has been an error in writing file PAXG
There has been an error in writing file WAVES
There has been an error in writing file SXP
There has been an error in writing file THETA
There has been an error in writing file DOT
There has been an error in writing file BAL
There has been an error in writing file LTC
There has been an error in writing file BCH
There has been an error in writing file XRP
There has been an error in writing file SOL
There has been an 

## writing spreads data for all perp-future pairs

In [4]:
acc.write_all_spreads(
    perp_folder_path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/all_perps',
    futures_folder_path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/expired_futures_data',
    output_path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/all_spreads')

Writing spread for 1INCH-0326
Writing spread for AAVE-0326
Writing spread for AAVE-1225
Writing spread for ADA-0326
Writing spread for ADA-0626
Writing spread for ADA-0925
Writing spread for ADA-1225
Writing spread for ADA-20191227
Writing spread for ADA-20200327
Writing spread for ALGO-0326
Writing spread for ALGO-0626
Writing spread for ALGO-0925
Writing spread for ALGO-1225
Writing spread for ALGO-20190927
Writing spread for ALGO-20191227
Writing spread for ALGO-20200327
Writing spread for ALT-0326
Writing spread for ALT-0626
Writing spread for ALT-0925
Writing spread for ALT-1225
Writing spread for ALT-20190927
Writing spread for ALT-20191227
Writing spread for ALT-20200327
Writing spread for ATOM-0326
Writing spread for ATOM-0626
Writing spread for ATOM-0925
Writing spread for ATOM-1225
Writing spread for ATOM-20191227
Writing spread for ATOM-20200327
Writing spread for AVAX-0326
Writing spread for AVAX-0925
Writing spread for AVAX-1225
Writing spread for BAL-0326
Writing spread f